# Distibert

In [1]:
import datasets

/home/kooler/dev/sw/sns-fake-content/model/training/transformer/env_transformer/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Fine Tune

In [2]:
dataset = datasets.load_from_disk("data/data_ds_m")
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label', '__index_level_0__'],
        num_rows: 51200
    })
    validation: Dataset({
        features: ['text', 'label', '__index_level_0__'],
        num_rows: 12800
    })
    test: Dataset({
        features: ['text', 'label', '__index_level_0__'],
        num_rows: 16000
    })
})

In [47]:
sum(map(lambda x: len(x.split(" ")),dataset['train']['text'])) / dataset['train'].num_rows

20.299609375

In [19]:
from transformers import BertTokenizer

tokenizer_id="bert-base-uncased"

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)


tokenized_datasets = dataset.map(tokenize_function, batched=True)
print(tokenized_datasets['train'][0])
tokenized_datasets

Loading cached processed dataset at /home/kooler/dev/sw/sns-fake-content/model/training/transformer/data/data_ds_m/train/cache-2364fce48e3bb007.arrow
Loading cached processed dataset at /home/kooler/dev/sw/sns-fake-content/model/training/transformer/data/data_ds_m/validation/cache-7767db1459af3f85.arrow
Loading cached processed dataset at /home/kooler/dev/sw/sns-fake-content/model/training/transformer/data/data_ds_m/test/cache-97b0f99ee456287e.arrow


{'text': 'borenmc: Another heated exchange, this time about whether an FBI investigation is warranted CNN The Lead With Jake Tapper  ', 'label': 1, '__index_level_0__': 60860, 'input_ids': [101, 8501, 2078, 12458, 1024, 2178, 9685, 3863, 1010, 2023, 2051, 2055, 3251, 2019, 8495, 4812, 2003, 10943, 2098, 13229, 1996, 2599, 2007, 5180, 11112, 4842, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

DatasetDict({
    train: Dataset({
        features: ['text', 'label', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 51200
    })
    validation: Dataset({
        features: ['text', 'label', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 12800
    })
    test: Dataset({
        features: ['text', 'label', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 16000
    })
})

In [23]:
import torch
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained("bert-base-uncased")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [10]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import evaluate

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

In [21]:
dataset['train']['text']

['borenmc: Another heated exchange, this time about whether an FBI investigation is warranted CNN The Lead With Jake Tapper  ',
 "Sarah Palin implied during her CPAC speech Karl Rove should run for office. Rove's reply: If I did &amp; won I'd serve my full term. #GOPBeefs",
 "Kourtney Kardashian is none too pleased by 'jealous' younger boyfriend Younes Bendjima ",
 "Selena Gomez Calls Justin Bieber Relationship 'Crazy And Very Intense': \n'It's uncomfortable that ever... ",
 '.@HaileeSteinfeld just shut down those @justinbieber dating rumors... read what she said here  ',
 'I liked a @YouTube video  Detroit : Become Human | Jericho ou es-tu ? | Episode 19',
 'Karrueche Tran Was “Terrified” of Running Into Chris Brown at BET Awards (Report)  ',
 'Katy Perry Literally Runs Away From Love–in Spite Of Her Ongoing Fling With Orlando\xa0Bloom  ',
 'Here Is The Real Reason Why Justin Bieber &amp; Selena Gomez Jetted Off To Seattle! - Gotta have faith! As we reported, over the weekend, Justin 

### Trainer

In [11]:
import time
import os

training_id = time.strftime(f"training-%Y%m%d-%H%M%S$-model-{model.name_or_path}")
training_id

'training-20230204-035311$-model-bert-base-uncased'

In [16]:
tokenized_datasets["train"].features

{'text': Value(dtype='string', id=None),
 'label': Value(dtype='int64', id=None),
 '__index_level_0__': Value(dtype='int64', id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'token_type_ids': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

In [13]:
assert(True)

In [14]:
from transformers import TrainingArguments, Trainer

# https://zhuanlan.zhihu.com/p/363670628
training_args = TrainingArguments(
    output_dir=f"./model/${training_id}/model/",
    evaluation_strategy="steps",
    prediction_loss_only=False,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=1,
    num_train_epochs=10,
    logging_dir=f"./model/${training_id}/log/",
    load_best_model_at_end=True,
    seed=2023
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    compute_metrics=compute_metrics,
)

In [15]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: __index_level_0__, text. If __index_level_0__, text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/home/kooler/dev/sw/sns-fake-content/model/env_model/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 51200
  Num Epochs = 10
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 32000
  Number of trainable parameters = 109483778


KeyboardInterrupt: 

In [ ]:
trainer